In [168]:
# Importações amplamente necessárias.
from pprint import pprint
import random as rd
from matplotlib import pyplot as pp
from numpy import concatenate, copy
from pygame.math import Vector2
from time import sleep

# Problema 1: Melhor Localização

A partir de um ponto específico, um punhado de elementos aleatórios na vizinhança deve convergir para o ponto.

Por simplicidade, faremos o problema em **duas dimensões** e, por causa disso, usaremos a classe Vector2 já criada dentro do _pygame.math_. Entretanto, nada impede que o usuário possa criar uma classe de Vector que possua N dimensões e coloque-a aqui.

## Criação de um ponto específico

In [169]:
def gerar_ponto_de_convergencia(
        dim: int = 2
) -> Vector2:
    """
    Descrição:
        Gera um ponto que a população deverá convergir.
    """
    return Vector2(
        [
            rd.random() for _ in range(dim)
        ]
    )

ponto_de_convergencia = gerar_ponto_de_convergencia()

## Criação de População Aleatória

In [170]:
def gerar_populacao_inicial(
        ponto_de_convergencia: Vector2,
        fator_de_distancia_do_ponto_de_convergencia: int = 2,
        fator_de_tamanho_da_nuvem: int = 0.5,
        quantidade_de_elementos_na_populacao: int = 50,
        dim: int = 2
) -> list[Vector2]:
    """
    Descrição:
        Gera uma nuvem de pontos aleatórios em torno de
        um ponto específico que representará a população inicial.

    Parâmetros:
        -> fator_de_distancia_do_ponto_de_convergencia:
            O quão longe a nuvem inicial estará do ponto de convergência.

        -> fator_de_tamanho_da_nuvem:
            O quão dispersa a nuvem estará.
    """

    ponto_aleatorio_central = ponto_de_convergencia + Vector2(
        [
            rd.random() for _ in range(dim)
        ]
    ) * fator_de_distancia_do_ponto_de_convergencia

    return list(
        ponto_aleatorio_central + Vector2(
            [
                rd.uniform(
                    -fator_de_tamanho_da_nuvem,
                    fator_de_tamanho_da_nuvem
                ) for _ in range(dim)
            ]
        ) for _ in range(quantidade_de_elementos_na_populacao)
    )

populacao_inicial = gerar_populacao_inicial(
    ponto_de_convergencia
)

## Forma de Apresentação

In [171]:
def plotar(
        ponto_de_convergencia: Vector2,
        lista_de_elementos: list[Vector2],
        index_do_melhor_elemento: int,
        geration: int
) -> None:
    """
    Descrição:
        Apresentará os pontos.
    """

    pp.scatter(
        ponto_de_convergencia.x,
        ponto_de_convergencia.y,
    )

    for index, elemento in enumerate(
        lista_de_elementos
    ):
        if index == index_do_melhor_elemento:
            pp.scatter(
                elemento.x,
                elemento.y,
                c = "red"
            )
        else:
            pp.scatter(
                elemento.x,
                elemento.y,
                c = "black"
            )

    pp.title(
        f"Apresentação Geração {geration}"
    )
    pp.xlim(
        0, 3
    )
    pp.ylim(
        0, 3
    )

    if geration % 10 == 0:
        pp.savefig(
            f"/content/retratos_de_geracoes/{geration}"
        )

    pp.show()

## Implementação

### Função Fitness: Melhor ⟶ Mais Perto

In [172]:
def obter_melhor(
        ponto_de_convergencia: Vector2,
        lista_de_elementos: list[Vector2]
) -> int:
    """
    Descrição:
        A partir da lista dos elementos da população, obterá o elemento
        mais perto e retornará seu index.

        Construimos o algoritmo na moral, apesar de podermos usarmos heap ou
        outros formas de otimização, pois pedir que o python faça tudo é criar
        dependência.
    """

    melhor_elemento = None
    menor_distancia_ate_o_momento = 10000  # Um número grande o suficiente que nunca será alcançado.


    for elemento in lista_de_elementos:
        distancia = (
            ponto_de_convergencia - elemento
        ).length_squared(
            # Dessa forma evitamos a raiz quadrado,
            # fato que apenas diminuiria performance.
        )

        if distancia <= menor_distancia_ate_o_momento:
            menor_distancia_ate_o_momento = distancia
            melhor_elemento = elemento

    return melhor_elemento

### Obter o Sortudo

In [173]:
def obter_sortudo(
        ponto_de_convergencia: Vector2,
        lista_de_elementos: list[Vector2],
        tamanho_da_populacao_de_possiveis_sortudos: int = None
) -> Vector2:
    """
    Descrição:
        Devemos obter um outro elemento, não necessariamente o 02,
        para que seja o segundo elemento a ser cruzado. Fazendo isso,
        permitimos a diversidade na população.
    """

    if tamanho_da_populacao_de_possiveis_sortudos is None:
        tamanho_da_populacao_de_possiveis_sortudos = len(
            lista_de_elementos
        ) // 10

    # Como a coleção é aleatória, tanto faz pegarmos elementos
    # aleatórios em posições aleatórias como sequenciais.

    return obter_melhor(
        ponto_de_convergencia,
        rd.sample(
            lista_de_elementos,
            tamanho_da_populacao_de_possiveis_sortudos
        )
    )

### Cruzamento

In [174]:
def fazer_sexo(
        elemento_1: Vector2,
        elemento_2: Vector2,
        dim: int = 2
) -> tuple[Vector2, Vector2]:
    """
    Descrição:
        'Cortaremos' cada genitor(Vetor) em partes
         e trocaremos elas para criar os filhos.

         A ideia é imaginar que, no caso geral, temos um
         vetor de n entradas. Escolhemos um filamento que
         vai do index a até o index b e trocaremos este
         filamento entre eles.
    """

    inicio_do_filamento_de_troca, final_do_filamento_de_troca = [
        rd.randint(
            0,
            dim - 1
        ) for _ in range(2)
    ]

    # Precisamos garantir que os index façam sentido.
    if inicio_do_filamento_de_troca > final_do_filamento_de_troca:
        inicio_do_filamento_de_troca, final_do_filamento_de_troca = final_do_filamento_de_troca, inicio_do_filamento_de_troca

    if inicio_do_filamento_de_troca == final_do_filamento_de_troca:
        final_do_filamento_de_troca += 1

    # Trocar os filamentos dos genitores
    temporario = copy(
        elemento_1
    )
    filho_1 = Vector2(
        list(
                concatenate(
                [
                    elemento_1[0 : inicio_do_filamento_de_troca],
                    elemento_2[inicio_do_filamento_de_troca : final_do_filamento_de_troca],
                    elemento_1[final_do_filamento_de_troca :]
                ]
            )
        )
    )

    filho_2 = Vector2(
        list(
                concatenate(
                [
                    elemento_2[0 : inicio_do_filamento_de_troca],
                    temporario[inicio_do_filamento_de_troca : final_do_filamento_de_troca],
                    elemento_2[final_do_filamento_de_troca : ]
                ]
            )
        )
    )

    return filho_1, filho_2

### Mutação

In [175]:
def mutar(
        elemento: Vector2,
        chance_de_mutacao: float = 0.7,
        escala_de_mutacao: float = 1,
        variables_de_mutabilidade: tuple[float, float] = (0.1, 0.1),
        dim: int = 2
) -> Vector2:
    """
    Descrição:
        Para obtermos novas possibilidades fora da capacidade
        da população atual, podemos realizar uma mutação.
    """

    # Verificando se irá ser mutado
    if rd.uniform(
        0,
        1
    ) > chance_de_mutacao:
        vetor_de_mutacao = Vector2(
            [
                # Vamos usar distribuição gamma.
                rd.gauss(
                    variables_de_mutabilidade[0],  # Média dos Valores de Mutação
                    variables_de_mutabilidade[1]  # Desvio Padrão dos Valores de Mutação
                ) for _ in range(len(elemento))
            ]
        )

        elemento = elemento + vetor_de_mutacao

    return elemento

### Atualizar Gerações

In [176]:
def atualizar_populacao(
        ponto_de_convergencia: Vector2,
        populacao_atual: list[Vector2],
        nova_populacao: list[Vector2],
        quantidade_de_elementos_possiveis_genitores: int,
        permitir_mutacao: bool,
        chance_de_mutacao: float,
        escala_de_mutacao: float,
        variables_de_mutabilidade: tuple[float, float]
) -> list[Vector2]:
    """
    Descrição:
        Gerará uma nova população a partir da anterior usando cruzamento
        dos 'melhores'.
    """
    total_population = len(
        populacao_atual
    )

    for index in range(
        total_population // 2
    ):
        genitor_1 = obter_sortudo(
            ponto_de_convergencia,
            populacao_atual,
            quantidade_de_elementos_possiveis_genitores
        )
        genitor_2 = obter_sortudo(
            ponto_de_convergencia,
            populacao_atual,
            quantidade_de_elementos_possiveis_genitores
        )

        # Perceba que se, por um acaso, forem iguais, apenas serão os mesmos.
        filho_1, filho_2 = fazer_sexo(
            genitor_1,
            genitor_2
        )

        # Agora acrescentar a mutação dentro da população.
        nova_populacao[
            index
        ] = mutar(
            filho_1,
            chance_de_mutacao,
            escala_de_mutacao,
            variables_de_mutabilidade
        ) if permitir_mutacao else filho_1

        nova_populacao[
            # irmão do cara de cima
            index + (total_population // 2)
        ] = mutar(
            filho_2,
            chance_de_mutacao,
            escala_de_mutacao,
            variables_de_mutabilidade
        ) if permitir_mutacao else filho_2

    return nova_populacao

### Deixando Em Condições de Uso

In [177]:
class Evolucao:
    def __init__(
            self,
            # Relativos à Criação da População Inicial
            quantidade_de_elementos_na_populacao: int,
            fator_de_distancia_do_ponto_de_convergencia: int,
            fator_de_tamanho_da_nuvem: int,
    ) -> None:

        self.ponto_de_convergencia = gerar_ponto_de_convergencia()

        self.tupla_de_geracoes = (
            gerar_populacao_inicial(
                self.ponto_de_convergencia,
                fator_de_distancia_do_ponto_de_convergencia,
                fator_de_tamanho_da_nuvem,
                quantidade_de_elementos_na_populacao
            ),
            [
                0 for _ in range(
                    # Apenas para termos uma segunda.
                    quantidade_de_elementos_na_populacao
                )
            ]
        )

        self.geracao = 0

    def proxima_geracao(
            self,
            se_deseja_plotagem: bool,
            quantidade_de_elementos_possiveis_genitores: int,
            permitir_mutacao: bool,
            chance_de_mutacao: float,
            escala_de_mutacao: float,
            variaveis_de_mutabilidade: tuple[float, float]
    ) -> None:

        atualizar_populacao(
            self.ponto_de_convergencia,
            self.tupla_de_geracoes[
                self.geracao % 2
            ],
            self.tupla_de_geracoes[
                (self.geracao + 1) % 2
            ],
            quantidade_de_elementos_possiveis_genitores,
            permitir_mutacao,
            chance_de_mutacao,
            escala_de_mutacao,
            variaveis_de_mutabilidade
        )

        self.geracao += 1

        if se_deseja_plotagem:

            plotar(
                self.ponto_de_convergencia,
                self.tupla_de_geracoes[
                    self.geracao % 2
                ],
                self.tupla_de_geracoes[
                    self.geracao % 2
                ].index(
                    obter_melhor(
                        self.ponto_de_convergencia,
                        self.tupla_de_geracoes[
                            self.geracao % 2
                        ]
                    )
                ),
                self.geracao
            )

### Executando

In [178]:
exemplo = Evolucao(
    50,  # quantidade de elementos na populacao
    2,  # fator de distancia do ponto de convergencia
    0.5  # fator de tamanho da nuvem
)

for geracao in range(
    # Máxima Geração
    20
):
    exemplo.proxima_geracao(
        True,
        10,  # quantidade de elementos possiveis genitores
        True,
        0.3,  # chance de cada filho ser mutado
        1,  # escala da mutacao
        (
            0.1, # Média das mutações
            0.1  # Desvio padrão das mutações
        )
    )

# Conforme Alteramos os resultados de mutabilidade, obtemos resultados bem diferentes.